# Тестирование в C++

In [71]:
#include <iostream>

using namespace std;

Unit-тестирование позволяет проверить на корректность выполнения определённые части программного продукта.

Здесь мы будем тестировать одну простую функцию:

In [72]:
int int_sum(int a, int b) {
    return a + b;
}

## I. Банальный способ

In [73]:
void banal_test(int a, int b, int result) {
    int res = int_sum(a, b);
    if (res != result) cout << "BAD (int_sum): " << result << " != " << res << "; a = " << a << ", b = " << b << endl;
}

In [74]:
banal_test(10, 20, 30);
banal_test(10, 20, 0);

BAD (int_sum): 0 != 30; a = 10, b = 20


### Минусы данного способа:

1) Необходимо писать новую функцию тестирования для другого компоненты программы;

2) Необходимо переписывать функцию при изменении объекта тестирования;

3) Вывод в тот же поток ostream;

## II. Использование шаблонов

In [75]:
template<typename RES>
void template_test(RES correct, RES func_result) {
    if (correct != func_result)
        cout << "BAD: " << correct << " != " << func_result << endl;
    else cout << "OK!" << endl;
}

In [76]:
template_test(30, int_sum(10, 20));
template_test(0, int_sum(10, 20));

OK!
BAD: 0 != 30


### Минусы данного способа:

1) Если функция принимает много параметров, то испортится красота кода;

2) Не самый информативный вывод;

3) Вывод также осуществляется в один поток ostream, вместе с программой;

### Плюсы данного способа:

1) Универсален (не нужно переписывать функцию тестирования)

Исправим вывод результатов тестов. Будем использовать другой поток вывода ostream.

Для этого импортируем библиотеку **stringstream** для вывода информации в данный поток и
создадим глобальную переменную для вывода.

In [77]:
#include <sstream>

ostringstream tests_output;

In [78]:
template<typename RES>
void template_test(RES correct, RES func_result) {
    if (correct != func_result)
        tests_output << "BAD: " << correct << " != " << func_result << '\n';
    else tests_output << "OK!\n";
}

In [79]:
template_test(10, int_sum(5, 5));
template_test(20, int_sum(5, 5));

Тогда вывод результатов тестов можно сделать в любое удобное для нас время.

In [80]:
cout << tests_output.str() << endl;

OK!
BAD: 20 != 10



Оформим вывод результатов тестов в функцию.

In [81]:
void print_tests_result() {
    cout << tests_output.str() << endl;
}

In [82]:
print_tests_result();

OK!
BAD: 20 != 10



Сделаем вывод функции информативнее.

Можно передавать id в функцию в качестве аргумента, 
но добавление лишних параметров только усложнит код, поэтому
сделаем автоматическую инкрементацию аргумента id.

In [91]:
tests_output.str("");
tests_output.clear();
// т. к. используем один поток вывода, но при разных реализациях тестов,
// то очистим поток вывода

In [92]:
template<typename RES>
void template_test(RES correct, RES func_result) {
    static int test_id = 0;
    test_id++;
    if (correct != func_result)
        tests_output << test_id << " BAD: " << correct << " != " << func_result << '\n';
    else tests_output << test_id << " OK!\n";
}

In [93]:
template_test(10, int_sum(5, 5));
template_test(20, int_sum(5, 5));

In [94]:
print_tests_result();

1 OK!
2 BAD: 20 != 10



## III. Шаблоны с передачей параметров

In [100]:
#include <tuple>
#include <utility>

In [101]:
tests_output.str("");
tests_output.clear();

In [111]:
template<typename CORRECT, typename FUNC, typename ...ARGS>
void test_unpacking(CORRECT correct, FUNC func, ARGS... args) {
    static int test_id = 0;
    test_id++;
    auto func_args = make_tuple(args...);
    CORRECT result = apply(func, func_args); // C++17+ only
    if (result != correct)
        tests_output << test_id << " BAD: " << correct << " != " << result << '\n';
    else tests_output << test_id << " OK!\n";
}

In [112]:
test_unpacking(20, int_sum, 10, 10);